In [2]:
from keras.layers import Input, Dense, LSTM, RepeatVector
from keras.models import Model
from keras.datasets import mnist
from keras.models import model_from_json
import numpy as np
# import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import time
from keras_tqdm import TQDMNotebookCallback
import logging
import json


Using TensorFlow backend.
Slow version of gensim.models.doc2vec is being used


In [6]:
word2vec_model = KeyedVectors.load_word2vec_format('lemmas.cbow.s100.w2v.bin', binary=True)

In [4]:
word2vec_model = None

In [3]:
sg_model = KeyedVectors.load_word2vec_format('lemmas.sg.s100.w2v.bin', binary=True)

In [1]:
word2vec_model

NameError: name 'word2vec_model' is not defined

In [4]:
sg_model

In [6]:
logger = logging.getLogger('experiments')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('autoencoder_experiments.log')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

def log(txt):
    print(txt)
    logger.info(txt)

In [7]:
input_dim = 100


In [5]:
datasets

NameError: name 'datasets' is not defined

In [8]:
words = [('joogitee', 'sõidutee'),
        ('õun', 'kivi'),
        ('countries', 'cities')]

datasets_sg = []

for word1, word2 in words:
    print(word1, word2)
    for window in [3,4]:
        for symmetric in [True]:
            print(window, symmetric)
            with open('../datasets-bak/contexts/{}_s_{}_w_{}.txt'.format(word1, symmetric, window)) as f:
                contexts1 = f.read().splitlines()
            with open('../datasets-bak/contexts/{}_s_{}_w_{}.txt'.format(word2, symmetric, window)) as f:
                contexts2 = f.read().splitlines()
            contexts = contexts1 + contexts2

            X = np.zeros((len(contexts), window*2, 100))
            for i, line in enumerate(contexts):
                X[i] = sg_model[line.split()]
            abbr = word1[0] + word2[0] + str(window)
            print(abbr)
            x_train, x_test = train_test_split(X, test_size=0.1, random_state=1)

            datasets_sg.append({'x_train':x_train, 'x_test': x_test, 'abbr': abbr})

joogitee sõidutee
3 True
js3
4 True
js4
õun kivi
3 True
õk3
4 True
õk4
countries cities
3 True
cc3
4 True
cc4


In [9]:
def log_data_json(architecture, timesteps, latent_dims):
#     return '{' + '"arch":{}, "timesteps":{}, "latent_dims":{}'.format(architecture, timesteps, latent_dims) + '}'
    return "a_{}_t_{}_d_{}".format(architecture, timesteps, latent_dims)

In [10]:
def arch1(timesteps, latent_dims):
    if len(latent_dims) != 1:
        raise Exception('incorrect latent dimensions count')
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dims[0])(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)
    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    sequence_autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')
    sequence_autoencoder.log_data = log_data_json(1, timesteps, latent_dims)
    return sequence_autoencoder

In [11]:
def arch2(timesteps, latent_dims):
    if len(latent_dims) != 2:
        raise Exception('incorrect latent dimensions count')
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dims[0], return_sequences=True)(inputs)
    encoded2 = LSTM(latent_dims[1], return_sequences=False)(encoded)
    decoded = RepeatVector(timesteps)(encoded2)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)
    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    sequence_autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')
    sequence_autoencoder.log_data = log_data_json(2, timesteps, latent_dims)
    return sequence_autoencoder

In [12]:
def arch3(timesteps, latent_dims):
    if len(latent_dims) != 2:
        raise Exception('incorrect latent dimensions count')
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dims[0], return_sequences=False)(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(latent_dims[1], return_sequences=True)(decoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)
    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    sequence_autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')
    sequence_autoencoder.log_data = log_data_json(3, timesteps, latent_dims)
    return sequence_autoencoder

In [13]:
def arch4(timesteps, latent_dims):
    if len(latent_dims) != 3:
        raise Exception('incorrect latent dimensions count')
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dims[0], return_sequences=True)(inputs)
    encoded2 = LSTM(latent_dims[1], return_sequences=False)(encoded)
    decoded = RepeatVector(timesteps)(encoded2)
    decoded = LSTM(latent_dims[2], return_sequences=True)(decoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)
    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    sequence_autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')
    sequence_autoencoder.log_data = log_data_json(4, timesteps, latent_dims)
    return sequence_autoencoder

In [17]:
def train(dataset, model, batch_size=64):
    
    sequence_autoencoder = model
    print('model log data', model.log_data)
    model_number = "sg_{}_{}_".format(int(time.time()-1491000000), dataset['abbr']) + model.log_data
    checkpointer = ModelCheckpoint(filepath="model-weights/{}.hdf5".format(model_number), verbose=0, save_best_only=True)
    json_string = sequence_autoencoder.to_json()
    with open('model-architectures/{}.json'.format(model_number), 'w') as f:
        f.write(json_string)

    sequence_autoencoder.fit(dataset['x_train'], dataset['x_train'],
                    epochs=500,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(dataset['x_test'], dataset['x_test']),
                    verbose=0,
                    callbacks=[TensorBoard(log_dir='training-logs/{}'.format(model_number), write_graph=False),
                                EarlyStopping(verbose=0, patience=10),
                                checkpointer,
                                TQDMNotebookCallback(leave_inner=False)
                              ])
#     return sequence_autoencoder


In [235]:
# train(dataset=datasets[4], model=arch, batch_size=64)

In [ ]:
latent_dim_choices = [[16], [32], [64], [128]]


for dataset in datasets_sg:
    print(dataset['abbr'])
    t = dataset['x_test'].shape[1]
    for latent_dim in latent_dim_choices:
        print('latent_dim', latent_dim)
        arch = arch1(t, latent_dim)
        train(dataset, arch)

js3
latent_dim [16]
model log data a_1_t_6_d_[16]


latent_dim [32]
model log data a_1_t_6_d_[32]


832/|/[loss: -0.549]  96%|| 832/864 [00:12<00:00, 40.86it/s]

latent_dim [64]
model log data a_1_t_6_d_[64]


latent_dim [128]
model log data a_1_t_6_d_[128]


...

cc4
latent_dim [16]
model log data a_1_t_8_d_[16]


latent_dim [32]
model log data a_1_t_8_d_[32]


4096/|/[loss: -0.503]  99%|| 4096/4152 [00:13<00:00, 690.71it/s]

latent_dim [64]
model log data a_1_t_8_d_[64]



latent_dim [128]
model log data a_1_t_8_d_[128]


In [19]:
latent_dim_choices = [[32, 32],
                        [64, 64],
                        [128 ,128],
                        [64, 32],
                        [128, 64]]

for dataset in datasets_sg:
    print(dataset['abbr'])
    t = dataset['x_test'].shape[1]
    for latent_dim in latent_dim_choices:
        print('latent_dim', latent_dim)
        arch = arch2(t, latent_dim)
        train(dataset, arch)

js3
latent_dim [32, 32]
model log data a_2_t_6_d_[32, 32]



latent_dim [64, 64]
model log data a_2_t_6_d_[64, 64]



latent_dim [128, 128]
model log data a_2_t_6_d_[128, 128]



latent_dim [64, 32]
model log data a_2_t_6_d_[64, 32]



latent_dim [128, 64]
model log data a_2_t_6_d_[128, 64]


...


cc4
latent_dim [32, 32]
model log data a_2_t_8_d_[32, 32]



latent_dim [64, 64]
model log data a_2_t_8_d_[64, 64]



latent_dim [128, 128]
model log data a_2_t_8_d_[128, 128]



latent_dim [64, 32]
model log data a_2_t_8_d_[64, 32]



latent_dim [128, 64]
model log data a_2_t_8_d_[128, 64]


In [1]:
122

122

In [ ]:
latent_dim_choices = [[32, 32],
                        [64, 64],
                        [128 ,128],
                        [32, 64],
                        [64, 128]]

for dataset in datasets_sg:
    print(dataset['abbr'])
    t = dataset['x_test'].shape[1]
    for latent_dim in latent_dim_choices:
        print('latent_dim', latent_dim)
        arch = arch3(t, latent_dim)
        train(dataset, arch)

js3
latent_dim [32, 32]
model log data a_3_t_6_d_[32, 32]



latent_dim [64, 64]
model log data a_3_t_6_d_[64, 64]



latent_dim [128, 128]
model log data a_3_t_6_d_[128, 128]



latent_dim [32, 64]
model log data a_3_t_6_d_[32, 64]



latent_dim [64, 128]
model log data a_3_t_6_d_[64, 128]


...


cc4
latent_dim [32, 32]
model log data a_3_t_8_d_[32, 32]



latent_dim [64, 64]
model log data a_3_t_8_d_[64, 64]



latent_dim [128, 128]
model log data a_3_t_8_d_[128, 128]



latent_dim [32, 64]
model log data a_3_t_8_d_[32, 64]



latent_dim [64, 128]
model log data a_3_t_8_d_[64, 128]


In [ ]:
latent_dim_choices = [[32, 32, 32],
                        [64, 64, 64],
                        [128, 128, 128],
                        [64, 32, 32],
                        [64, 64, 32],
                        [128, 64, 32]]

for dataset in datasets_sg:
    print(dataset['abbr'])
    t = dataset['x_test'].shape[1]
    for latent_dim in latent_dim_choices:
        print('latent_dim', latent_dim)
        arch = arch4(t, latent_dim)
        train(dataset, arch)

js3
latent_dim [32, 32, 32]
model log data a_4_t_6_d_[32, 32, 32]



latent_dim [64, 64, 64]
model log data a_4_t_6_d_[64, 64, 64]



latent_dim [128, 128, 128]
model log data a_4_t_6_d_[128, 128, 128]



latent_dim [64, 32, 32]
model log data a_4_t_6_d_[64, 32, 32]



latent_dim [64, 64, 32]
model log data a_4_t_6_d_[64, 64, 32]


...


latent_dim [64, 64, 64]
model log data a_4_t_8_d_[64, 64, 64]



latent_dim [128, 128, 128]
model log data a_4_t_8_d_[128, 128, 128]



latent_dim [64, 32, 32]
model log data a_4_t_8_d_[64, 32, 32]



latent_dim [64, 64, 32]
model log data a_4_t_8_d_[64, 64, 32]



latent_dim [128, 64, 32]
model log data a_4_t_8_d_[128, 64, 32]


In [295]:
from tensorflow.python.summary import event_accumulator as ea
import tensorflow as tf

In [301]:
for e in tf.train.summary_iterator("training-logs/3777514_js3/events.out.tfevents.1494777517.vegan"):
    for v in e.summary.value:
        print(v)
#         if v.tag == 'loss' or v.tag == 'accuracy':
#             print(v.simple_value)
#     print(dir(e))

tag: "val_loss"
simple_value: -1.9211703538894653

tag: "loss"
simple_value: -1.1492760181427002

tag: "val_loss"
simple_value: -2.2888426780700684

tag: "loss"
simple_value: -2.0620570182800293

tag: "val_loss"
simple_value: -2.4448766708374023

tag: "loss"
simple_value: -2.3127601146698

tag: "val_loss"
simple_value: -2.532550096511841

tag: "loss"
simple_value: -2.429046630859375

tag: "val_loss"
simple_value: -2.6073341369628906

tag: "loss"
simple_value: -2.519144296646118

tag: "val_loss"
simple_value: -2.6847622394561768

tag: "loss"
simple_value: -2.6113603115081787

tag: "val_loss"
simple_value: -2.747035026550293

tag: "loss"
simple_value: -2.673861026763916

tag: "val_loss"
simple_value: -2.689533233642578

tag: "loss"
simple_value: -2.7056190967559814

tag: "val_loss"
simple_value: -2.701730489730835

tag: "loss"
simple_value: -2.6677043437957764

tag: "val_loss"
simple_value: -2.75380539894104

tag: "loss"
simple_value: -2.6801583766937256

tag: "val_loss"
simple_value: -2

In [293]:
acc = ea.EventAccumulator("training-logs/3777514_js3/")
acc.Reload()

# Print tags of contained entities, use these names to retrieve entities as below
print(acc.Tags())

# E. g. get all values and steps of a scalar called 'l2_loss'
xy_l2_loss = [(s.step, s.value) for s in acc.Scalars('l2_loss')]



INFO:tensorflow:No path found after training-logs/3777514_js3/events.out.tfevents.1494777517.vegan
{'compressedHistograms': [], 'meta_graph': False, 'images': [], 'graph': False, 'audio': [], 'histograms': [], 'scalars': ['val_loss', 'loss'], 'run_metadata': []}


KeyError: 'Key l2_loss was not found in Reservoir'

### Testing code

In [207]:
datasets[0]['x_test'].shape

(96, 6, 100)

In [194]:
auto = train(datasets[4], latent_dim=128, batch_size=64)

In [41]:
auto = train(datasets[0], latent_dim=128, batch_size=64)

In [145]:
encoder1 = Model(auto.layers[0].input, auto.layers[2].output)

In [270]:
with open('model-architectures/3442138.json', 'r') as f:
    json = f.read()

model = model_from_json(json)
model.load_weights('model-weights/3442138.hdf5')
encoder2 = Model(model.layers[0].input, model.layers[2].output)

In [273]:
arch = arch1(6, [32])
# encoder2.predict(x_test[:1])

In [274]:
lulz = datasets[4]['x_train']

In [275]:
lulz.shape

(5382, 6, 100)

In [278]:
arch.predict(lulz[:1]).shape

(1, 6, 100)

In [279]:
encoder3 = Model(arch.layers[0].input, arch.layers[2].output)

In [280]:
encoder3.predict(lulz[:1]).shape

(1, 6, 32)

In [155]:
np.allclose(encoder1.predict(x_test[:1]), encoder2.predict(x_test[:1]))

False

In [253]:
# encoder1.predict(x_test[:1]), encoder2.predict(x_test[:1])